# 01 — Explore Boundary Data

Use this notebook to load and inspect the enumeration area boundaries and control grid files from Google Drive.

**What this notebook covers:**
- Loading project configuration
- Reading boundary GeoJSON files from Google Drive
- Inspecting schema, CRS, and geometry types
- Quick visualization of boundaries

## Setup

Add the project root to the Python path so we can import our `src` modules.

In [ ]:
# Standard library
import sys
from pathlib import Path

# Third-party
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as cx

# Add project root to path
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Local imports (after path setup)
from src.utils.config_loader import load_config, get_data_dir, get_output_dir
from src.data_processing.load_boundaries import build_control_grid, save_control_grid

# Configuration
config = load_config()
DATA_DIR = get_data_dir(config)
OUTPUT_DIR = get_output_dir(config)

print(f"Project root: {PROJECT_ROOT}")
print(f"Data exists:  {DATA_DIR.exists()}")
print(f"Data dir:     {DATA_DIR}")
print(f"Output dir:   {OUTPUT_DIR}")


The config loader reads `config/config.yaml` and merges any local overrides from `config/config.local.yaml`. This way your local Google Drive path stays out of version control.

## Explore Study Area File

Load the Study_Area_30km.geojson and inspect what's inside — columns, CRS, geometry types, and a quick plot.

In [ ]:
# Load the study area file
grid_5km_path = DATA_DIR / "01_input_data" / "boundaries" / "Area_study_5km_grid.gpkg"
grid_5km = gpd.read_file(grid_5km_path)

print(f"Shape:          {grid_5km.shape}")
print(f"CRS:            {grid_5km.crs}")
print(f"Geometry types: {grid_5km.geom_type.unique()}")
print(f"\nColumns: {list(grid_5km.columns)}")
print(f"\nDtypes:\n{grid_5km.dtypes}")
grid_5km.head(10)


In [ ]:
#merge with control areas sampled
control_sampled_path = DATA_DIR / "01_input_data" / "boundaries" / "Rubeho_control_areas_sampled.csv"
control_sampled = pd.read_csv(control_sampled_path)
print(control_sampled.shape)

control_sampled.head()
control_sampled.control_pixel_sampled.value_counts(dropna=False
                                                       )

print("grid_5km columns:", list(grid_5km.columns))
print("sampled columns:", list(control_sampled.columns))

#merge sampled areas with flags
# Only keep control areas (sampled=1 or replacement=0)
control_ids = control_sampled[control_sampled["control_pixel_sampled"].isin([0, 1])]
id_to_status = control_ids.set_index("grid_id")["control_pixel_sampled"].map({1: "sampled", 0: "replacement"})

grid_5km["sample_status"] = grid_5km["id"].map(id_to_status)

print(grid_5km["sample_status"].value_counts(dropna=False))
grid_5km.head(10)

# Verify counts match source data
assert (grid_5km["sample_status"] == "sampled").sum() == (control_sampled["control_pixel_sampled"] == 1).sum(), "Sampled count mismatch"
assert (grid_5km["sample_status"] == "replacement").sum() == (control_sampled["control_pixel_sampled"] == 0).sum(), "Replacement count mismatch"

print("Counts match")
print(grid_5km["sample_status"].value_counts(dropna=False))



grid_5km

In [ ]:
print(grid_5km['id'].value_counts(dropna=False))


In [ ]:
grid_5km_wm = grid_5km.to_crs(epsg=3857)

fig, ax = plt.subplots(1, 1, figsize=(12, 10))

# All grid cells as background
grid_5km_wm.plot(ax=ax, facecolor="none", edgecolor="#CCCCCC", linewidth=0.5)

# Sampled controls
grid_5km_wm[grid_5km_wm["sample_status"] == "sampled"].plot(
    ax=ax, facecolor="blue", edgecolor="blue", linewidth=0.8, alpha=0.4, label="Sampled"
)

# Replacements
grid_5km_wm[grid_5km_wm["sample_status"] == "replacement"].plot(
    ax=ax, facecolor="orange", edgecolor="orange", linewidth=0.8, alpha=0.4, label="Replacement"
)

import contextily as cx
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik)

ax.set_title("5km Grid — Sampled vs Replacement Controls")
ax.set_axis_off()
plt.tight_layout()
plt.show()


## Load control grid cells
The control grid areas file should be at:
```
<data_dir>/01_input_data/boundaries/control_grid_5km_flagged.gpkg
```

In [ ]:

control_grid = build_control_grid(DATA_DIR)
save_control_grid(control_grid, DATA_DIR)
control_grid.head()





## Load full grid if neccesary

The control grid is the set of cells that each enumerator will use for navigation.

In [ ]:
full_grid = gpd.read_file(DATA_DIR / "01_input_data" / "boundaries" / "study_area_5km_flagged.gpkg")
full_grid.head()

# Quick script to generate smaller grid cells (500m by 500m) based on existing auxiliary data. 
Plot enumeration areas and control grid cells together to verify the data looks correct.

In [ ]:
# src = r"G:\Shared drives\TZ-CCT_RUBEV-0825\Data\Data\0.1_geospatial data\aux_shapefile\grid_500m_5km_ID.shp"

# # Build a WHERE clause for  the 48 control IDs AND the treatment IDs, 
# control_ids = control_grid["id"].astype(int).tolist()
# id_list = ",".join(str(i) for i in control_ids)
# where_clause = f'"5km_id" IN ({id_list}) OR "is_treatme" = \'True\''

# subgrid = gpd.read_file(src, engine="pyogrio", where=where_clause)

# print(f"Shape: {subgrid.shape}")
# print(f"Unique 5km IDs: {subgrid['5km_id'].nunique()}")
# print(f"is_treatme values: {subgrid['is_treatme'].unique()}")
# subgrid.head(10)


In [ ]:
# output_path = DATA_DIR / "01_input_data/boundaries/subgrid_500m_control_treatment.gpkg"
# subgrid.to_file(output_path, driver="GPKG")
# print(f"Saved {len(subgrid)} rows to {output_path}")



# Load the saved 500m subgrid


In [ ]:
subgrid = gpd.read_file(DATA_DIR / "01_input_data/boundaries/subgrid_500m_control_treatment.gpkg")
subgrid_treatment = subgrid[subgrid["is_treatme"] == "True"]
control_ids = set(control_grid["id"].astype(int))

subgrid_control = subgrid[subgrid["5km_id"].isin(control_ids)]

#note geometry are points, not polygons. 

#generating grid cells instead of points. 

In [ ]:
subgrid_control.head()
subgrid_control.crs
subgrid_control.cell_size.unique()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))

# 500m control sub-grid
subgrid_control.to_crs(epsg=3857).plot(
    ax=ax, facecolor="none", edgecolor="blue", linewidth=0.4
)

# 5km control grid overlay
control_grid.to_crs(epsg=3857).plot(
    ax=ax, facecolor="none", edgecolor="red", linewidth=2
)

cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik)

ax.set_axis_off()
ax.set_title(f"Control 500m Sub-grids ({len(subgrid_control)} cells) with 5km Grid Overlay", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()


In [ ]:
test_5km_id = control_grid["id"].iloc[1]
cell_5km = control_grid[control_grid["id"] == test_5km_id]
cells_500m = subgrid_control[subgrid_control["5km_id"] == int(test_5km_id)]

fig, ax = plt.subplots(figsize=(10, 10))

cells_500m.to_crs(epsg=3857).plot(ax=ax, facecolor="none", edgecolor="blue", linewidth=1)
cell_5km.to_crs(epsg=3857).plot(ax=ax, facecolor="none", edgecolor="red", linewidth=2)

cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik, zoom=14)

ax.set_axis_off()
ax.set_title(f"5km cell {test_5km_id} — 500m sub-grid", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()



In [ ]:
fig, ax = plt.subplots(figsize=(14, 14), dpi=150)

cells_500m_merc = cells_500m.to_crs(epsg=3857)
cell_5km_merc = cell_5km.to_crs(epsg=3857)

# Plot boundary explicitly
cells_500m_merc.boundary.plot(ax=ax, color="blue", linewidth=0.8)
cell_5km_merc.boundary.plot(ax=ax, color="red", linewidth=2.5)

cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik, zoom=14)

ax.set_axis_off()
ax.set_title(f"5km cell {test_5km_id} — 500m sub-grid", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()


In [ ]:
print(cells_500m.geom_type.unique())
print(cells_500m.geometry.iloc[0])
